In [1]:
import pandas as pd
import os
import pyarrow
import openpyxl

import os
import pyarrow
import openpyxl
import pickle
from typing import Dict

from exgauster.utils import DataOfExgauster
from dataclasses import dataclass
from typing import Optional
import exgauster.trained_booster

@dataclass
class TrainExgausterConfig:
    hyperparametres: dict
    window_size: int
    booster: Optional[exgauster.trained_booster.TrainedBooster]
    booster_scoring: Optional[exgauster.trained_booster.TrainedBooster]


with open('final_trained_boosters_with_test_score 4.0-8.5.pkl', "rb") as f:
    exgauster_map = pickle.load(f)


C:\tools\miniconda3\envs\stal\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
with open('final_trained_boosters_with_test_score8-9.pkl', "rb") as f:
    exgauster_map_8_9 = pickle.load(f)



In [3]:
with open('prepared_data.pkl', "rb") as f:
    exgausters_data = pickle.load(f)



In [10]:
# test_data = exgausters_data[4].test

In [4]:

from merlion.utils import TimeSeries


In [27]:
exg_index = forecasts_m1_with_future[0].index

def make_forecasts_on_test(m_key='m3', forecasts_of_future=None):
    """

    :param m_key: m1 или m3
    :param forecasts_of_future: прогнозы на месяц вперед по рядам
    :return: датафреймы с прогнозами по фичам
    """
    result_forecasts = []
    exg_numbers = [4, 5, 6, 7, 8, 9]
    for exg_number in exg_numbers:
        test_data = exgausters_data[exg_number].test
        for feature_num in range(len(exgauster_map[exg_number][m_key])):
            print('make forecast for exgauster', exg_number, m_key, 'feature', feature_num, )
            if (int(exg_number) == 8 and int(feature_num) >= 6) or int(exg_number) == 9:
                booster = exgauster_map_8_9[exg_number][m_key][feature_num].booster
            else:
                booster = exgauster_map[exg_number][m_key][feature_num].booster

            if booster is not None:
                if forecasts_of_future is not None:
                    # добавить к данным тестирования построенные прогнозы
                    x_for_forecast = pd.concat([test_data, forecasts_of_future[exg_number]])
                else:
                    x_for_forecast = test_data

                fc, err = booster.model.forecast(booster.x_train, TimeSeries.from_pd(x_for_forecast).align())
                result_forecasts.append(fc)
            else:
                if m_key == 'm3':
                    y = exgausters_data[exg_number].y.m3[feature_num]
                else:
                    y = exgausters_data[exg_number].y.m1[feature_num]

                if y.all_is0:
                    df = pd.DataFrame(index=exg_index, columns=[y.colname], data=0)
                    result_forecasts.append(df)
                elif y.all_is1:
                    df = pd.DataFrame(index=exg_index, columns=[y.colname], data=1)
                    result_forecasts.append(df)
                else:
                    print('error: exgauster ', exg_number, m_key, 'feature: ', feature_num)
                    df = pd.DataFrame(index=exg_index, columns=[y.colname], data=0)
                    result_forecasts.append(df)
    return result_forecasts



In [ ]:
forecasts_m1 = make_forecasts_on_test('m1')
forecasts_m3 = make_forecasts_on_test('m3')


In [6]:
def join_forecasts_to_df(forecasts, index=None):
    if index is None:
        index = forecasts[0].index
    new_df = pd.DataFrame(index=index)
    for df in forecasts:
        new_df = new_df.merge(df, left_index=True, right_index=True)
    return new_df

In [ ]:
joined_m1 = join_forecasts_to_df(forecasts_m1)
joined_m3 = join_forecasts_to_df(forecasts_m3)

joined_m1_resampled = joined_m1.resample('10S').asfreq().fillna(method='ffill').round(decimals=0)
joined_m3_resampled = joined_m3.resample('10S').asfreq().fillna(method='ffill').round(decimals=0)


In [71]:
joined_m1.to_parquet('m1_answers_noresampled.parquet')
joined_m3.to_parquet('m3_answers_noresampled.parquet')


In [59]:
joined_m1_mask = joined_m1 == 1
joined_m3_replaced = joined_m3.replace(1, 2)
joined_m3_unioned_m1 = joined_m3_replaced.where(~joined_m1_mask, 1)

joined_m1_resampled_mask = joined_m1_resampled == 1
joined_m3_resampled_replaced = joined_m3_resampled.replace(1, 2)
joined_m3_resampled_unioned_m1 = joined_m3_resampled_replaced.where(~joined_m1_resampled_mask, 1)



In [60]:
print(0, joined_m3_unioned_m1.values.ravel().tolist().count(0))
print(1, joined_m3_unioned_m1.values.ravel().tolist().count(1))
print(2, joined_m3_unioned_m1.values.ravel().tolist().count(2))

print(0, joined_m3_resampled_unioned_m1.values.ravel().tolist().count(0))
print(1, joined_m3_resampled_unioned_m1.values.ravel().tolist().count(1))
print(2, joined_m3_resampled_unioned_m1.values.ravel().tolist().count(2))


0 22303014
1 2618
2 1027993
0 669085664
1 78540
2 30839471


In [61]:
# ответ на задачу 1 в ожидаемом формате + форматы по отдельности
joined_m3_resampled_unioned_m1.to_parquet('submission_2.parquet')
joined_m1_resampled.to_parquet('submission_2_only_M1==1.parquet')
joined_m3_resampled_replaced.to_parquet('submission_2_only_M3==2.parquet')



In [8]:
forecasts_list = {}

for idx in [4, 5, 6, 7, 8, 9]:
    fc = pd.read_parquet(os.path.join('forecasts', f'forecast_exgauster{idx}.parquet'))
    forecasts_list[idx] = fc


for idx in []:
    fc = pd.read_parquet(os.path.join('forecasts', f'forecast_exgauster{idx}.parquet'))
    fc = fc.filter(like='_y')
    fc = fc.rename(columns=lambda x: x.rstrip('_y'))
    forecasts_list[idx] = fc


In [10]:
len(forecasts_list[9].columns)

16

In [12]:
print(forecasts_list[9].index[0], forecasts_list[9].index[-1])

2023-05-10 01:00:00
2023-04-10 00:00:00 2023-05-10 01:00:00


In [13]:
forecasts_list_resampled = {}

for idx, df in forecasts_list.items():
     resampled_df = df.resample('5Min').asfreq().fillna(method='ffill')
     fc = resampled_df.drop(resampled_df.index[0])
     forecasts_list_resampled[idx] = fc


In [84]:
test_data = exgausters_data[4].test

In [83]:
type(test_data)

pandas.core.frame.DataFrame

In [85]:
new_df1 = pd.concat([test_data, forecasts_list_resampled[4]])

In [87]:
new_df1

,ЭКСГАУСТЕР 4. ТОК РОТОРА 1,ЭКСГАУСТЕР 4. ТОК РОТОРА2,ЭКСГАУСТЕР 4. ТОК СТАТОРА,ЭКСГАУСТЕР 4. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2,ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3,ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4,ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 1,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 2,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
2022-01-01 00:00:00,247.496667,247.496667,271.203667,134.441000,42.553103,43.135000,30.913793,34.47500,31.359615,32.620526,0.723000,1.188000,0.342069,1.903793,1.810333,1.361000
2022-01-01 00:05:00,247.510000,247.510000,270.190667,134.345667,42.703000,43.081000,30.884000,34.55600,31.559000,32.612000,0.726667,1.194667,0.315000,1.860333,1.853667,1.368333
2022-01-01 00:10:00,247.524333,247.524333,268.929667,134.279000,42.757000,43.351000,30.911000,34.63700,31.640000,32.774000,0.748333,1.200333,0.313667,1.904000,1.719667,1.372333
2022-01-01 00:15:00,247.497667,247.497667,268.851000,134.135667,42.703000,43.135000,31.208000,34.69100,31.721000,33.044000,0.758333,1.214333,0.324667,1.876667,1.770333,1.343333
2022-01-01 00:20:00,247.422667,247.422667,269.148667,134.011667,42.811000,43.027000,31.316000,34.71800,31.694000,33.044000,0.780333,1.218333,0.358000,1.825000,1.853667,1.395333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 23:40:00,245.507655,245.248751,128.731640,105.131238,41.385403,26.153676,27.556355,31.36450,22.867082,30.592384,2.005687,1.159304,0.897693,1.656713,1.833309,0.604046
2023-04-30 23:45:00,245.507655,245.248751,128.731640,105.131238,41.385403,26.153676,27.556355,31.36450,22.867082,30.592384,2.005687,1.159304,0.897693,1.656713,1.833309,0.604046
2023-04-30 23:50:00,245.507655,245.248751,128.731640,105.131238,41.385403,26.153676,27.556355,31.36450,22.867082,30.592384,2.005687,1.159304,0.897693,1.656713,1.833309,0.604046
2023-04-30 23:55:00,245.507655,245.248751,128.731640,105.131238,41.385403,26.153676,27.556355,31.36450,22.867082,30.592384,2.005687,1.159304,0.897693,1.656713,1.833309,0.604046


In [28]:

forecasts_m1_with_future = make_forecasts_on_test('m1', forecasts_list_resampled)
# forecasts_m3_with_future = make_forecasts_on_test('m3', forecasts_list_resampled)




make forecast for exgauster 4 m1 feature 0


C:\devs\ts\LTC2023_15\exgauster\ts_transformer_for_lgbm.py:62: RuntimeWarning: invalid value encountered in cast
  labels = labels_to_fit[window_diff:].ravel().round().astype(np.int32).astype(np.float64)


make forecast for exgauster 4 m1 feature 1
make forecast for exgauster 4 m1 feature 2
make forecast for exgauster 4 m1 feature 3


C:\devs\ts\LTC2023_15\exgauster\ts_transformer_for_lgbm.py:62: RuntimeWarning: invalid value encountered in cast
  labels = labels_to_fit[window_diff:].ravel().round().astype(np.int32).astype(np.float64)


make forecast for exgauster 4 m1 feature 4
make forecast for exgauster 4 m1 feature 5


...

C:\devs\ts\LTC2023_15\exgauster\ts_transformer_for_lgbm.py:62: RuntimeWarning: invalid value encountered in cast
  labels = labels_to_fit[window_diff:].ravel().round().astype(np.int32).astype(np.float64)


make forecast for exgauster 8 m1 feature 22
make forecast for exgauster 8 m1 feature 23
make forecast for exgauster 8 m1 feature 24
make forecast for exgauster 8 m1 feature 25
make forecast for exgauster 8 m1 feature 26
make forecast for exgauster 8 m1 feature 27
make forecast for exgauster 8 m1 feature 28
make forecast for exgauster 8 m1 feature 29
make forecast for exgauster 8 m1 feature 30
make forecast for exgauster 9 m1 feature 0
make forecast for exgauster 9 m1 feature 1
make forecast for exgauster 9 m1 feature 2
make forecast for exgauster 9 m1 feature 3
make forecast for exgauster 9 m1 feature 4
make forecast for exgauster 9 m1 feature 5
make forecast for exgauster 9 m1 feature 6
make forecast for exgauster 9 m1 feature 7
make forecast for exgauster 9 m1 feature 8
make forecast for exgauster 9 m1 feature 9


C:\devs\ts\LTC2023_15\exgauster\ts_transformer_for_lgbm.py:62: RuntimeWarning: invalid value encountered in cast
  labels = labels_to_fit[window_diff:].ravel().round().astype(np.int32).astype(np.float64)


make forecast for exgauster 9 m1 feature 10
make forecast for exgauster 9 m1 feature 11
make forecast for exgauster 9 m1 feature 12
make forecast for exgauster 9 m1 feature 13
make forecast for exgauster 9 m1 feature 14
make forecast for exgauster 9 m1 feature 15
make forecast for exgauster 9 m1 feature 16
make forecast for exgauster 9 m1 feature 17
make forecast for exgauster 9 m1 feature 18
make forecast for exgauster 9 m1 feature 19
make forecast for exgauster 9 m1 feature 20
make forecast for exgauster 9 m1 feature 21
make forecast for exgauster 9 m1 feature 22
make forecast for exgauster 9 m1 feature 23
make forecast for exgauster 9 m1 feature 24
make forecast for exgauster 9 m1 feature 25
make forecast for exgauster 9 m1 feature 26
make forecast for exgauster 9 m1 feature 27


C:\devs\ts\LTC2023_15\exgauster\ts_transformer_for_lgbm.py:62: RuntimeWarning: invalid value encountered in cast
  labels = labels_to_fit[window_diff:].ravel().round().astype(np.int32).astype(np.float64)


In [32]:

joined_with_future_m1 = join_forecasts_to_df(forecasts_m1_with_future)
joined_with_future_m3 = join_forecasts_to_df(forecasts_m3_with_future)


In [30]:
forecasts_m1_with_future[0].index[-1]


Timestamp('2023-05-10 01:00:00', freq='5T')

In [33]:
joined_with_future_m1.to_parquet('m1_answers_with_future_noresampled.parquet')
joined_with_future_m3.to_parquet('m3_answers_with_future_noresampled.parquet')


In [25]:
joined_with_future_m1.index[-1]

Timestamp('2023-04-10 00:00:00', freq='5T')

In [17]:
test_exgauster_num = 9
test_booster = exgauster_map_8_9[test_exgauster_num]['m1'][9].booster
test_test_data = exgausters_data[test_exgauster_num].test


In [25]:
test_test_data.index[-1]

Timestamp('2023-04-10 00:00:00', freq='5T')

In [20]:
forecasts_list_resampled[test_exgauster_num].index[-1]


Timestamp('2023-05-10 01:00:00', freq='5T')

In [21]:
test_x_for_forecast.index[-1]


Timestamp('2023-05-10 01:00:00', freq='5T')

In [ ]:
ts_test = TimeSeries.from_pd(test_test_data).align()
ts_test

In [22]:

test_x_for_forecast = pd.concat([test_test_data, forecasts_list_resampled[test_exgauster_num]])

fc, err = test_booster.model.forecast(test_booster.x_train, TimeSeries.from_pd(test_x_for_forecast).align())


C:\devs\ts\LTC2023_15\exgauster\ts_transformer_for_lgbm.py:62: RuntimeWarning: invalid value encountered in cast
  labels = labels_to_fit[window_diff:].ravel().round().astype(np.int32).astype(np.float64)


In [23]:
fc.index[-1]

Timestamp('2023-05-10 01:00:00', freq='5T')

In [32]:
len(test_x_for_forecast.columns)

48

In [33]:
len(forecasts_list_resampled[test_exgauster_num].columns)

32

In [34]:
len(test_test_data.columns)

16

In [36]:
for idx, colname in enumerate(test_test_data.columns):
    print(colname == forecasts_list_resampled[test_exgauster_num].columns[idx], colname, forecasts_list_resampled[test_exgauster_num].columns[idx], )

False ЭКСГАУСТЕР 9. ТОК РОТОРА 1 ЭКСГАУСТЕР 9. ТОК РОТОРА 1_x
False ЭКСГАУСТЕР 9. ТОК РОТОРА 2 ЭКСГАУСТЕР 9. ТОК РОТОРА 2_x
False ЭКСГАУСТЕР 9. ТОК СТАТОРА ЭКСГАУСТЕР 9. ТОК СТАТОРА_x
False ЭКСГАУСТЕР 9. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ ЭКСГАУСТЕР 9. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ_x
False ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1 ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1_x
False ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2 ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2_x
False ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3 ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3_x
False ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4 ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4_x
False ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ_x
False ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ_x
False ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 1 ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 1_x
False ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 2 ЭКСГАУСТ

In [38]:
for i in range(4,10):
    for idx, colname in enumerate(forecasts_list_resampled[i].columns):
        print(i, idx, colname)


4 0 ЭКСГАУСТЕР 4. ТОК РОТОРА 1
4 1 ЭКСГАУСТЕР 4. ТОК РОТОРА2
4 2 ЭКСГАУСТЕР 4. ТОК СТАТОРА
4 3 ЭКСГАУСТЕР 4. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ
4 4 ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1
4 5 ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2
4 6 ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3
4 7 ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4
4 8 ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ
4 9 ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ
4 10 ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 1
4 11 ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 2
4 12 ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3
4 13 ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.
4 14 ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4
4 15 ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
5 0 ЭКСГАУСТЕР 5. ТОК РОТОРА 1
5 1 ЭКСГАУСТЕР 5. ТОК РОТОРА 2
5 2 ЭКСГАУСТЕР 5. ТОК СТАТОРА
5 3 ЭКСГАУСТЕР 5. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ
5 4 ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1
5 5 ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2
5 6 ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3
5 7 ЭКСГАУСТ

In [39]:
data = pd.read_parquet(os.path.join('forecasts', f'forecast_exgauster{9}.parquet'))


In [41]:
data = data.filter(like='_y')
data = data.rename(columns=lambda x: x.rstrip('_y'))

print(len(data.columns))

16
